In [6]:
# import MongoDB modules
from pymongo import MongoClient

# import the Requests HTTP library
import requests

# import the Beautiful Soup module 
from bs4 import BeautifulSoup

# import the time module for the sleep functionality
# in order to be polite when scraping
import time

# randomize a bit for the scraping
import random

In [7]:
# connect to the hosted MongoDB instance
client = MongoClient('mongodb://localhost:27017/')

# connect to our mongodb indie game database collection
db = client.capstone

#indie game DB raw web scrape for users

raw_user_scrape = db.raw_user_scrape

In [8]:
def insert(collection, dictionary):
    if not collection.find_one({"user": dictionary["user"]}):
        try:
            collection.insert_one(dictionary)
            print "inserted", dictionary["user"]
            
        except Exception, e:
            print e
            
    else:
        print "already exists"

In [9]:
db.raw_user_scrape.find().count()

8511

In [10]:
path = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=5DA6749271B3262B589F2980379B9AE2&steamids='

user_id = "76561197960435530"

req = requests.get(path + user_id)

In [11]:
print req

<Response [200]>


In [12]:
#'76561197900035535' & 0xFFFFFFFF

In [13]:
#profile_path = "https://steamcommunity.com/id/{}/games/?tab=all".format(good_id)
profile_path = "https://steamcommunity.com/id/{}/games/?tab=all".format("stas350")

error_path = "https://steamcommunity.com/id/{}/games/?tab=all".format("henko")

good_path = "https://steamcommunity.com/id/{}/games/?tab=all".format("Dart")

print good_path

good_req = requests.get(good_path)
bad_req = requests.get(error_path)

https://steamcommunity.com/id/Dart/games/?tab=all


In [14]:
profile_error = BeautifulSoup(bad_req.content, "lxml")

profile_good = BeautifulSoup(good_req.content, "lxml")

In [15]:
print len(profile_error.prettify())

print len(profile_good.prettify())

112052
151714


In [16]:
if len(profile_good.prettify()) < 21000:
    print "error page"
else:
    print "not error page"
    print
    games_list = profile_good.find(id="games_list_rows")
    
    # make dict for mongo_db
    mongo_dict = {
        "user":"Dart",
        "data":profile_good.prettify()        
    }
    
    insert(raw_user_scrape, mongo_dict)

not error page

already exists


In [199]:
print profile_soup

<!DOCTYPE html>
<html class=" responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Steam Community :: Aebleskive :: Games</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://steamcommunity-a.akamaihd.net/public/shared/css/motiva_sans.css?v=qIG_iZC1Vj_d" rel="stylesheet" type="text/css"/>
<link href="https://steamcommunity-a.akamaihd.net/public/shared/css/buttons.css?v=DivqMdJb8hoZ" rel="stylesheet" type="text/css"/>
<link href="https://steamcommunity-a.akamaihd.net/public/shared/css/shared_global.css?v=TBcFCzFI_Hwx" rel="stylesheet" type="text/css"/>
<link href="https://steamcommunity-a.akamaihd.net/public/css/globalv2.css?v=fhcxF1tVnmT1" rel="stylesheet" type="text/css"/>
<link href="https://steamcommunity-a.akamaihd.net/public/css/skin_1/profilev2.css?v=zzmEDDRQfJIU" rel="styl

In [200]:
soup = BeautifulSoup(req.content, "lxml")

In [201]:
para = soup.find_all("p")

In [202]:
str(para).find("personaname")
169

print str(para)[str(para).find("personaname"):].split(",")[0]
# personaname": "Robin"

print str(para)[str(para).find("personaname"):].split(",")[0].split(" ")[1].strip('"')
# Robin

personaname": "Robin"
Robin


In [203]:
def attempt_to_get_user_profile(user_id, path, bad_soup):
    
    # make real request
    good_req = requests.get(path)
    
    # convert to soup object
    profile_good = BeautifulSoup(good_req.content, "lxml")
    
    # see if we got the error page
    if len(profile_good.prettify()) < 21000:
        print "error page for", user_id
    else:

        # make dict for mongo_db
        mongo_dict = {
            "user":user_id,
            "data":profile_good.prettify()        
        }

        insert(raw_user_scrape, mongo_dict)

In [ ]:
path = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=5DA6749271B3262B589F2980379B9AE2&steamids='

user_id =   "76561197900000000"
user_id = "765611979"
# user_id = "76561197960435530" # valid
# user_id = "765611979468871517" # invalid

# API profile: this represents an empty response, if we get this then it's not a good profile, try again
none_soup = BeautifulSoup('<html><body><p>{\n\t"response": {\n\t\t"players": [\n\n\t\t]\n\t\t\n\t}\n}</p></body></html>', "lxml")

# User web profile, intentionally generate an error/empty profile so we can skip storing that
error_path = "https://steamcommunity.com/id/{}/games/?tab=all".format("stas350")
bad_req = requests.get(error_path)

# this is what we will check against to make sure that the response isn't an error
profile_error = BeautifulSoup(bad_req.content, "lxml")

# path for generating a request will be used like this
# good_path = "https://steamcommunity.com/id/{}/games/?tab=all".format("Dart")
good_path = "https://steamcommunity.com/id/{}/games/?tab=all"

# attempt to avoid pointlessly repeating requests
tried_ids = []

# keep track of how many profiles attempted
profiles_attempted = []

for idx in xrange(45000):
    user_id = "765611979"
    user_num = ""
    
    # generate random numbers for user_id to try
    for idy in xrange(8):
        user_num += str(random.randint(0, 9))
     
    # try to be nice to steam servers
    time.sleep(random.random() * 1.0)
    
    # coalesce the user_id
    user_id = user_id + user_num
    
    # try to prevent duplicate requests
    if user_id not in tried_ids:
        # add id so it will show in request checks
        tried_ids.append(user_id)

        #print user_id

        req = requests.get(path + user_id)

        soup = BeautifulSoup(req.content, "lxml")

        if soup != none_soup:
            profiles_attempted.append(user_id)
            
            
            print "Attempt:", idx, "profiles attempted/found so far:", len(profiles_attempted), "/", len(tried_ids)
            print req, "user_id:", user_id

            # find the username in the profile info stuff
            para = soup.find_all("p")
            user_id = str(para)[str(para).find("personaname"):].split(",")[0].split(" ")[1].strip('"')
            attempt_to_get_user_profile(user_id, good_path.format(user_id), profile_error)
            print
            
        else:
            pass

Attempt: 1 profiles attempted/found so far: 1 / 2
<Response [200]> user_id: 76561197964188928
error page for \u041d\u043e\u0432\u043e\u0440\u043e\u0441\u0441\u0438\u044f

Attempt: 11 profiles attempted/found so far: 2 / 12
<Response [200]> user_id: 76561197969355626
error page for RheeneilSSJG

Attempt: 17 profiles attempted/found so far: 3 / 18
<Response [200]> user_id: 76561197966901619
error page for bromii-_-

Attempt: 24 profiles attempted/found so far: 4 / 25
<Response [200]> user_id: 76561197989611862
error page for mopesyjr1

Attempt: 31 profiles attempted/found so far: 5 / 32
<Response [200]> user_id: 76561197970937159
error page for octave.ardiet

Attempt: 32 profiles attempted/found so far: 6 / 33
<Response [200]> user_id: 76561197960979054
error page for the_13th_apostle

Attempt: 34 profiles attempted/found so far: 7 / 35
<Response [200]> user_id: 76561197966780657
error page for youdokjon33

Attempt: 36 profiles attempted/found so far: 8 / 37
<Response [200]> user_id: 765

Attempt: 322 profiles attempted/found so far: 67 / 323
<Response [200]> user_id: 76561197989749732
error page for lauracssd

Attempt: 331 profiles attempted/found so far: 68 / 332
<Response [200]> user_id: 76561197995669384
inserted DeSSaR

Attempt: 336 profiles attempted/found so far: 69 / 337
<Response [200]> user_id: 76561197999870934
error page for ultra-fed

Attempt: 339 profiles attempted/found so far: 70 / 340
<Response [200]> user_id: 76561197967844770
error page for powerzone2k

Attempt: 341 profiles attempted/found so far: 71 / 342
<Response [200]> user_id: 76561197985907163
inserted Lilou

Attempt: 347 profiles attempted/found so far: 72 / 348
<Response [200]> user_id: 76561197960871055
error page for fallenaznable

Attempt: 358 profiles attempted/found so far: 73 / 359
<Response [200]> user_id: 76561197990874964
error page for paulohenrique18

Attempt: 364 profiles attempted/found so far: 74 / 365
<Response [200]> user_id: 76561197976547961
error page for Dintchiclets

Atte

inserted put2up

Attempt: 584 profiles attempted/found so far: 133 / 585
<Response [200]> user_id: 76561197996416383
inserted Nina

Attempt: 585 profiles attempted/found so far: 134 / 586
<Response [200]> user_id: 76561197963463865
error page for kelso0119

Attempt: 591 profiles attempted/found so far: 135 / 592
<Response [200]> user_id: 76561197991862941
inserted Paladin

Attempt: 595 profiles attempted/found so far: 136 / 596
<Response [200]> user_id: 76561197961731602
inserted Squat

Attempt: 597 profiles attempted/found so far: 137 / 598
<Response [200]> user_id: 76561197963028009
inserted DoD

Attempt: 605 profiles attempted/found so far: 138 / 606
<Response [200]> user_id: 76561197977322727
error page for McBiggies

Attempt: 608 profiles attempted/found so far: 139 / 609
<Response [200]> user_id: 76561197987100084
inserted CarloAntonio

Attempt: 612 profiles attempted/found so far: 140 / 613
<Response [200]> user_id: 76561197982123794
inserted Lucian

Attempt: 614 profiles attemp

Attempt: 894 profiles attempted/found so far: 200 / 895
<Response [200]> user_id: 76561197990782802
inserted Will

Attempt: 895 profiles attempted/found so far: 201 / 896
<Response [200]> user_id: 76561197997198365
error page for iso_damla

Attempt: 905 profiles attempted/found so far: 202 / 906
<Response [200]> user_id: 76561197998686926
error page for chaabane.benzait

Attempt: 911 profiles attempted/found so far: 203 / 912
<Response [200]> user_id: 76561197992240840
error page for weslleysilva89

Attempt: 914 profiles attempted/found so far: 204 / 915
<Response [200]> user_id: 76561197994233565
error page for atom.bulat

Attempt: 917 profiles attempted/found so far: 205 / 918
<Response [200]> user_id: 76561197979581876
error page for Styles`

Attempt: 918 profiles attempted/found so far: 206 / 919
<Response [200]> user_id: 76561197986783048
error page for analord-09

Attempt: 923 profiles attempted/found so far: 207 / 924
<Response [200]> user_id: 76561197979473556
error page for ho

Attempt: 1220 profiles attempted/found so far: 265 / 1221
<Response [200]> user_id: 76561197971321671
error page for cabioman

Attempt: 1225 profiles attempted/found so far: 266 / 1226
<Response [200]> user_id: 76561197980769217
error page for fawkinna

Attempt: 1231 profiles attempted/found so far: 267 / 1232
<Response [200]> user_id: 76561197965867393
inserted monkey

Attempt: 1234 profiles attempted/found so far: 268 / 1235
<Response [200]> user_id: 76561197980863240
inserted Longhorn

Attempt: 1235 profiles attempted/found so far: 269 / 1236
<Response [200]> user_id: 76561197999905127
error page for Schmauzk\xf6nig.

Attempt: 1237 profiles attempted/found so far: 270 / 1238
<Response [200]> user_id: 76561197960627618
error page for zweefkees

Attempt: 1238 profiles attempted/found so far: 271 / 1239
<Response [200]> user_id: 76561197996634530
error page for ced_rocha

Attempt: 1249 profiles attempted/found so far: 272 / 1250
<Response [200]> user_id: 76561197995591251
error page fo

...

inserted Ickarus

Attempt: 6445 profiles attempted/found so far: 1438 / 6445
<Response [200]> user_id: 76561197993060190
inserted Kreminho

Attempt: 6448 profiles attempted/found so far: 1439 / 6448
<Response [200]> user_id: 76561197985066266
error page for the_evil_sens

Attempt: 6453 profiles attempted/found so far: 1440 / 6453
<Response [200]> user_id: 76561197989425417
inserted nizN

Attempt: 6459 profiles attempted/found so far: 1441 / 6459
<Response [200]> user_id: 76561197995626637
error page for prgjr83159

Attempt: 6464 profiles attempted/found so far: 1442 / 6464
<Response [200]> user_id: 76561197990766590
inserted Lord

Attempt: 6465 profiles attempted/found so far: 1443 / 6465
<Response [200]> user_id: 76561197990411469
inserted HeavyWeight

Attempt: 6470 profiles attempted/found so far: 1444 / 6470
<Response [200]> user_id: 76561197981855549
error page for whatsthot

Attempt: 6471 profiles attempted/found so far: 1445 / 6471
<Response [200]> user_id: 76561197991870268
erro

inserted admin

Attempt: 6720 profiles attempted/found so far: 1503 / 6720
<Response [200]> user_id: 76561197990413251
error page for jeje1944

Attempt: 6721 profiles attempted/found so far: 1504 / 6721
<Response [200]> user_id: 76561197979068367
error page for dream_girlshe

Attempt: 6730 profiles attempted/found so far: 1505 / 6730
<Response [200]> user_id: 76561197991498814
error page for boyspsg91

Attempt: 6734 profiles attempted/found so far: 1506 / 6734
<Response [200]> user_id: 76561197994688086
error page for waynesghetto

Attempt: 6742 profiles attempted/found so far: 1507 / 6742
<Response [200]> user_id: 76561197963071010
error page for sabsi14

Attempt: 6744 profiles attempted/found so far: 1508 / 6744
<Response [200]> user_id: 76561197963668707
error page for .....

Attempt: 6748 profiles attempted/found so far: 1509 / 6748
<Response [200]> user_id: 76561197992855228
error page for fler9000

Attempt: 6765 profiles attempted/found so far: 1510 / 6765
<Response [200]> user_i

error page for wlindberg

Attempt: 7044 profiles attempted/found so far: 1568 / 7044
<Response [200]> user_id: 76561197961223151
error page for CarolinaGirl

Attempt: 7048 profiles attempted/found so far: 1569 / 7048
<Response [200]> user_id: 76561197983074991
inserted rocketman111

Attempt: 7050 profiles attempted/found so far: 1570 / 7050
<Response [200]> user_id: 76561197987928060
error page for jump_nigga

Attempt: 7052 profiles attempted/found so far: 1571 / 7052
<Response [200]> user_id: 76561197983383818
error page for cesinhapr

Attempt: 7056 profiles attempted/found so far: 1572 / 7056
<Response [200]> user_id: 76561197985048211
error page for Arkhaem

Attempt: 7058 profiles attempted/found so far: 1573 / 7058
<Response [200]> user_id: 76561197990598916
inserted tsquare

Attempt: 7069 profiles attempted/found so far: 1574 / 7069
<Response [200]> user_id: 76561197991942493
error page for rekoitim

Attempt: 7072 profiles attempted/found so far: 1575 / 7072
<Response [200]> user_

Attempt: 7358 profiles attempted/found so far: 1633 / 7358
<Response [200]> user_id: 76561197984634594
inserted maxi

Attempt: 7360 profiles attempted/found so far: 1634 / 7360
<Response [200]> user_id: 76561197992447087
error page for yanglovexia

Attempt: 7361 profiles attempted/found so far: 1635 / 7361
<Response [200]> user_id: 76561197993120778
error page for marvin9_

Attempt: 7362 profiles attempted/found so far: 1636 / 7362
<Response [200]> user_id: 76561197962260857
inserted Dread

Attempt: 7368 profiles attempted/found so far: 1637 / 7368
<Response [200]> user_id: 76561197987708955
inserted Lucas

Attempt: 7369 profiles attempted/found so far: 1638 / 7369
<Response [200]> user_id: 76561197994902038
error page for tedi-95

Attempt: 7395 profiles attempted/found so far: 1639 / 7395
<Response [200]> user_id: 76561197980596481
inserted cooldude50

Attempt: 7396 profiles attempted/found so far: 1640 / 7396
<Response [200]> user_id: 76561197997084885
inserted h3ad?!

Attempt: 7410 

Attempt: 7633 profiles attempted/found so far: 1698 / 7633
<Response [200]> user_id: 76561197985873721
inserted Smoke

Attempt: 7635 profiles attempted/found so far: 1699 / 7635
<Response [200]> user_id: 76561197989091433
error page for =CV=wolsnkilla

Attempt: 7639 profiles attempted/found so far: 1700 / 7639
<Response [200]> user_id: 76561197986525003
error page for guettnix

Attempt: 7640 profiles attempted/found so far: 1701 / 7640
<Response [200]> user_id: 76561197987926532
error page for janysik1

Attempt: 7642 profiles attempted/found so far: 1702 / 7642
<Response [200]> user_id: 76561197994589047
error page for fomara350

Attempt: 7653 profiles attempted/found so far: 1703 / 7653
<Response [200]> user_id: 76561197996984145
error page for razor_drilon

Attempt: 7655 profiles attempted/found so far: 1704 / 7655
<Response [200]> user_id: 76561197990368353
error page for shoopdewoop

Attempt: 7657 profiles attempted/found so far: 1705 / 7657
<Response [200]> user_id: 76561197991611

In [26]:
"{:02d}".format(2)

'02'

In [17]:
raw_user_scrape.find().count()
# raw_user_scrape.drop()

8511

In [18]:
users = []

for document in raw_user_scrape.find():
    #print type(document)
    #print len(document)
    
    users.append(document["user"])

In [19]:
print users[:10]

[u'Santyaga', u'zaeb', u'FokeR', u'Augmonger', u'Medic', u'n0name', u'Mulligan', u'Clowny', u'maschine', u'Diglett']


In [46]:
doc = raw_user_scrape.find_one(["user", "maschine"])

In [47]:
print type(doc)

<type 'NoneType'>


In [120]:
doc = raw_user_scrape.find_one()

In [121]:
type(doc)

dict

In [122]:
print doc["user"]

Santyaga


In [141]:
def get_userinfo_from_doc(document):
    '''
    Extracts the user info from the user all games owned website into a dictionary with the username as one
    key/value and then a list of user_game_data as the user_game_data value.
    
    
    Ex:  {"user": "bob", user_game_data: [{game_1}, {game_2"}, {etc}]}
    Ex: game_1 = {"game_name":_name, "app_id":_appid, "hours_played":_hours_played, "last_played":_last_played}
    
    returns:
    the dictionary specified above
    '''
    data = document["data"].lower()
    
    # remove useless front part of data
    data = data[data.index("var rggames = [".lower()):(3+data.index("}}];".lower()))]
    
    # attempt to split the block of game info into sub-blocks specific to each game
    raw_user_game_data = data.split("},{")
    
    # be overly explicit in where each variable will be stored which will assemble
    # the dictionary specific for each user's game playtime, name, etc
#     _name = ""
#     _appid = ""
#     _hours_played = ""
#     _last_played = ""
    
    # make a list to hold the individual game info dictionaries
    game_info = []
    
    # raw_user_data is a list of game data so walk over that and process each
    # item into a dictionary to be added to the overall user profile
    for raw_game_info in raw_user_game_data:
        
        # split the individual raw_game_info into each specific attribute
        raw_game_attributes = raw_game_info.split(",")
        
        # walk all of the attributes
        for item in raw_game_attributes:
#             print attribute
            
            if "appid" in item:
                _appid = item.split(":")[1]

            if "name" in item:
                _name = item[7:]

            if "hours_forever" in item:
                _hours_played = float(item.split(":")[1].strip('"'))

            if "last_played" in item:
                _last_played = int(item.split(":")[1])

        this_game_info_dict = {"game_name":_name, "app_id":_appid, "hours_played":_hours_played, "last_played":_last_played}
        #print
        game_info.append(this_game_info_dict)
        
        #print this_game_info_dict
        
        #print sub_info
    
    # make the total player game data dictionary:
    player_dict = {"player":document["user"], "game_data": game_info}
    
    # return to caller
    return player_dict

In [144]:
player = get_userinfo_from_doc(doc)

for game in player["game_data"]:
    print game

{'hours_played': 109.0, 'last_played': 1481709465, 'game_name': u'"s.t.a.l.k.e.r.: call of pripyat"', 'app_id': u'41700'}
{'hours_played': 92.0, 'last_played': 1441473783, 'game_name': u'"tropico 4"', 'app_id': u'57690'}
{'hours_played': 75.0, 'last_played': 1465502884, 'game_name': u'"s.t.a.l.k.e.r.: clear sky"', 'app_id': u'20510'}
{'hours_played': 38.0, 'last_played': 1436302037, 'game_name': u'"dead island"', 'app_id': u'91310'}
{'hours_played': 35.0, 'last_played': 1497815346, 'game_name': u'"mafia iii"', 'app_id': u'360430'}
{'hours_played': 35.0, 'last_played': 1325232000, 'game_name': u'"tropico 3 - steam special edition"', 'app_id': u'23490'}
{'hours_played': 31.0, 'last_played': 1378472188, 'game_name': u'"s.t.a.l.k.e.r.: shadow of chernobyl"', 'app_id': u'4830'}
{'hours_played': 29.0, 'last_played': 1404329465, 'game_name': u'"half-life 2"', 'app_id': u'220'}
{'hours_played': 26.0, 'last_played': 1497297927, 'game_name': u'"dishonored 2"', 'app_id': u'403640'}
{'hours_played

In [86]:
data = doc["data"].lower()

# remove useless front part of data
data = data[data.index("var rggames = [".lower()):(3+data.index("}}];".lower()))]

In [104]:
# print data.index("Tropico".lower())
# print data.index("Pripyat".lower())

# print data.index("var rggames = [".lower())



# print data.index("}}];".lower())

# print data[:500]

# var rggames = [{"appid":41700,"name":"s.t.a.l.k.e.r.: call of pripyat","logo":"https:\/\/steamcdn-a.akamaihd.net\/steamcommunity\/public\/images\/apps\/41700\/93b0cfaad1110993eff5f2017d37511c3e5258a5.jpg","friendlyurl":false,"availstatlinks":{"achievements":false,"global_achievements":false,"stats":false,"leaderboards":false,"global_leaderboards":false},"hours_forever":"109","last_played":1481709465},{"appid":57690,"name":"tropico 4","logo":"https:\/\/steamcdn-a.akamaihd.net\/steamcommunity\/pub

info = data.split("},{")
_name = ""
_appid = ""
_hours_played = ""
_last_played = ""
sub_info = info[0].split(",")
for item in sub_info:
    if "appid" in item:
        _appid = item.split(":")[1]

    if "name" in item:
        _name = item[7:]
        
    if "hours_forever" in item:
        _hours_played = float(item.split(":")[1].strip('"'))
        
    if "last_played" in item:
        _last_played = int(item.split(":")[1])
        
info_dict = {"game_name":_name, "app_id":_appid, "hours_played":_hours_played, "last_played":_last_played}
print
print info_dict
print
print sub_info


{'hours_played': 109.0, 'last_played': 1481709465, 'game_name': u'"s.t.a.l.k.e.r.: call of pripyat"', 'app_id': u'41700'}

[u'var rggames = [{"appid":41700', u'"name":"s.t.a.l.k.e.r.: call of pripyat"', u'"logo":"https:\\/\\/steamcdn-a.akamaihd.net\\/steamcommunity\\/public\\/images\\/apps\\/41700\\/93b0cfaad1110993eff5f2017d37511c3e5258a5.jpg"', u'"friendlyurl":false', u'"availstatlinks":{"achievements":false', u'"global_achievements":false', u'"stats":false', u'"leaderboards":false', u'"global_leaderboards":false}', u'"hours_forever":"109"', u'"last_played":1481709465']


In [64]:
stroink = '''l_hover" onmouseout="hideachievementhover( this, event, $('global_hover') )" style="position:absolute; display: none; left: 0px; top: 0px;">
      <div class="shadow_ul">
      </div>
      <div class="shadow_top">
      </div>
      <div class="shadow_ur">
      </div>
      <div class="shadow_left">
      </div>
      <div class="shadow_right">
      </div>
      <div class="shadow_bl">
      </div>
      <div class="shadow_bottom">
      </div>
      <div class="shadow_br">
      </div>
      <div class="hover_box shadow_content">
       <div class="content" id="global_hover_content">
       </div>
      </div>
      <div class="hover_arrow">
      </div>
     </div>
     <div id="uninstallconfirm" onclick="confirmcancel(event)">
      <div class="uninstallconfirmbutton" id="uninstallconfirmyes">
       yes, uninstall
      </div>
      <div class="uninstallconfirmbutton">
       no, cancel
      </div>
     </div>
     <script language="javascript">'''

In [65]:
len(stroink)

967